In [ ]:
!pip install import_ipynb
!pip install segmentation-models
!pip install tensorflow==2.8
!pip install keras==2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 899.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 45.0 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstall

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import import_ipynb
%run /content/drive/MyDrive/Segmentation/UpdatedMeanIoU.ipynb
#import unet
#from unet import build_unet

In [ ]:
#from unet import build_unet
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.metrics import MeanIoU
from keras.callbacks import EarlyStopping
import tensorflow as tf
import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()

Segmentation Models: using `keras` framework.


'tf.keras'

In [ ]:
#New generator with rotation and shear where interpolation that comes with rotation and shear are thresholded in masks.
#This gives a binary mask rather than a mask with interpolated values.
seed=24
batch_size= 8
n_classes= 9

In [ ]:
img_data_gen_args = dict(rescale = 1/255., #Original pixel values are 0 and 255. So rescaling to 0 to 1
                      width_shift_range=0.3,
                      height_shift_range=0.3,
                      shear_range=0.5,
                      horizontal_flip=True,
                      vertical_flip=True,
                      fill_mode='reflect')

In [ ]:
#If You need to resize images then add this to the flow_from_directory parameters
#target_size=(150, 150), #Or whatever the size is for your network

image_data_generator = ImageDataGenerator(**img_data_gen_args)
image_generator = image_data_generator.flow_from_directory("/content/drive/MyDrive/Segmentation/dataset/train_frame/",
                                                           seed=seed,
                                                           batch_size=batch_size,
                                                           class_mode=None)  #Very important to set this otherwise it returns multiple numpy arrays
                                                                            #thinking class mode is binary.

mask_generator = image_data_generator.flow_from_directory("/content/drive/MyDrive/Segmentation/dataset/train_mask/",
                                                         seed=seed,
                                                         batch_size=batch_size,
                                                         color_mode = 'grayscale',   #Read masks in grayscale
                                                         class_mode=None)


valid_img_generator = image_data_generator.flow_from_directory("/content/drive/MyDrive/Segmentation/dataset/val_frame/",
                                                               seed=seed,
                                                               batch_size=batch_size,
                                                               class_mode=None) #Default batch size 32, if not specified here
valid_mask_generator = image_data_generator.flow_from_directory("/content/drive/MyDrive/Segmentation/dataset/val_mask/",
                                                               seed=seed,
                                                               batch_size=batch_size,
                                                               color_mode = 'grayscale',   #Read masks in grayscale
                                                               class_mode=None)  #Default batch size 32, if not specified here




Found 9705 images belonging to 1 classes.
Found 9705 images belonging to 1 classes.
Found 3338 images belonging to 1 classes.
Found 3338 images belonging to 1 classes.


In [ ]:
test_img_generator = image_data_generator.flow_from_directory("/content/drive/MyDrive/Segmentation/dataset/test_frame/",
                                                              seed=seed,
                                                              batch_size=32,
                                                              class_mode=None) #Default batch size 32, if not specified here

test_mask_generator = image_data_generator.flow_from_directory("/content/drive/MyDrive/Segmentation/dataset/test_mask/",
                                                              seed=seed,
                                                              batch_size=32,
                                                              color_mode = 'grayscale',   #Read masks in grayscale
                                                              class_mode=None)  #Default batch size 32, if not specified here


Found 3252 images belonging to 1 classes.
Found 3252 images belonging to 1 classes.


In [ ]:
from keras import backend as K

#Dice metric can be a great metric to track accuracy of semantic segmentation.
def dice_metric(y_pred, y_true):
    intersection = K.sum(K.sum(K.abs(y_true * y_pred), axis=-1))
    union = K.sum(K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1))
    # if y_pred.sum() == 0 and y_pred.sum() == 0:
    #     return 1.0

    return 2*intersection / union


In [ ]:
# preprocess input
BACKBONE = 'resnet152'
preprocess_input = sm.get_preprocessing(BACKBONE)

image_generator = preprocess_input(image_generator)
mask_generator = preprocess_input(mask_generator)
valid_img_generator = preprocess_input(valid_img_generator)
valid_mask_generator = preprocess_input(valid_mask_generator)
test_img_generator = preprocess_input(test_img_generator)
test_mask_generator = preprocess_input(test_mask_generator)

train_generator = zip(image_generator,mask_generator)
val_generator = zip(valid_img_generator, valid_mask_generator)
test_generator = zip(test_img_generator, test_mask_generator)

model = sm.FPN(BACKBONE, classes=n_classes, encoder_weights='imagenet')

234340352/234326024 [==============================] - 1s 0us/step


In [ ]:
mIOU = UpdatedMeanIoU(num_classes=9)

#num_train_imgs = len(os.listdir('/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/train_frame/frame/'))
#num_train_mask = len(os.listdir('/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/train_mask/mask/'))

num_train_imgs = len(os.listdir('/content/drive/MyDrive/Segmentation/dataset/train_frame/frame/'))
num_train_mask = len(os.listdir('/content/drive/MyDrive/Segmentation/dataset/train_mask/mask/'))

print(num_train_imgs, num_train_mask)
steps_per_epoch = num_train_imgs //batch_size
print(steps_per_epoch)

model.compile(optimizer=Adam(learning_rate = 1e-3), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=[mIOU, dice_metric])
early_stopping = EarlyStopping(monitor="val_loss", patience=5, verbose=1, restore_best_weights=True,)

history=model.fit(train_generator, validation_data=val_generator, steps_per_epoch=steps_per_epoch,
                  validation_steps=steps_per_epoch, epochs=20,  callbacks=[early_stopping],)

9705 9705
1213


/usr/local/lib/python3.10/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/2


In [ ]:
print(test_img_generator[0].shape)
num_test_imgs = len(test_img_generator[0])

print(num_test_imgs)
batch_size = 16

steps_per_epoch = num_test_imgs //batch_size
print(steps_per_epoch)

_, dice_metric, mIOU = model1.evaluate(test_generator, steps=steps_per_epoch)
print("Dice metric is = ", (dice_metric * 100.0), "%", "mIOU is = ", (mIOU * 100.0), "%")

In [ ]:
#model.save('semanticSegmentation_50epochs.hdf5')
#model = tf.keras.models.load_model("semanticSegmentation_50epochs.hdf5", compile=False)